In [4]:
import os
import pandas as pd
import exifread
from hachoir.parser import createParser
from hachoir.metadata import extractMetadata
from fractions import Fraction

def get_exif_data(image_path):
    with open(image_path, 'rb') as img_file:
        tags = exifread.process_file(img_file)
        exif_data = {
            "file_name": os.path.basename(image_path),
            "latitude": None,
            "longitude": None,
            "altitude": None,
            "date_time": None,
        }

        # Extract GPS data if available
        gps_latitude = tags.get("GPS GPSLatitude")
        gps_latitude_ref = tags.get("GPS GPSLatitudeRef")
        gps_longitude = tags.get("GPS GPSLongitude")
        gps_longitude_ref = tags.get("GPS GPSLongitudeRef")
        gps_altitude = tags.get("GPS GPSAltitude")
        date_time_original = tags.get("EXIF DateTimeOriginal")

        def convert_to_degrees(value):
            parts = str(value).strip('[]').replace(' ', '').split(',')
            d = float(Fraction(parts[0]))
            m = float(Fraction(parts[1]))
            s = float(Fraction(parts[2]))
            return d + (m / 60.0) + (s / 3600.0)

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = convert_to_degrees(gps_latitude)
            lon = convert_to_degrees(gps_longitude)

            exif_data["latitude"] = lat * (-1 if gps_latitude_ref.values[0] == 'S' else 1)
            exif_data["longitude"] = lon * (-1 if gps_longitude_ref.values[0] == 'W' else 1)
        
        if gps_altitude:
            exif_data["altitude"] = float(gps_altitude.values[0])

        if date_time_original:
            exif_data["date_time"] = str(date_time_original)

    return exif_data

def get_video_metadata(video_path):
    parser = createParser(video_path)
    metadata = extractMetadata(parser)
    video_data = {
        "file_name": os.path.basename(video_path),
        "latitude": None,
        "longitude": None,
        "altitude": None,
        "date_time": None,
    }

    if metadata:
        for item in metadata.exportPlaintext():
            if "Creation date" in item:
                video_data["date_time"] = item.split(": ")[1].strip()
            if "GPS coordinates" in item:
                coords = item.split(": ")[1].strip().split(", ")
                lat, lon = coords[0].split(" "), coords[1].split(" ")
                video_data["latitude"] = float(lat[0]) * (-1 if lat[1] == 'S' else 1)
                video_data["longitude"] = float(lon[0]) * (-1 if lon[1] == 'W' else 1)

    return video_data

def process_files(folder):
    files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.mov'))]
    data_list = []

    for file in files:
        file_path = os.path.join(folder, file)
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            data = get_exif_data(file_path)
        elif file.lower().endswith('.mov'):
            data = get_video_metadata(file_path)
        data_list.append(data)

    return data_list

# Ruta de la carpeta de fotos y videos
folder_path = 'C:/Users/34670/Desktop/python/coast_to_coast/viaje_usa/fotos_usa'

# Procesar archivos y crear DataFrame
data_list = process_files(folder_path)
df_fotos_videos = pd.DataFrame(data_list)

# Mostrar el DataFrame
df_fotos_videos.head()


PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.


,file_name,latitude,longitude,altitude,date_time
0,IMG_3600.JPG,40.646500,-73.789667,NaN,2013:07:28 19:02:00
1,IMG_3603.JPG,40.703667,-74.097000,NaN,2013:07:29 10:05:23
2,IMG_3605.JPG,40.690667,-74.042500,2.444101,2013:07:29 10:07:50
3,IMG_3607.JPG,40.690167,-74.042667,6.688098,2013:07:29 10:08:16
4,IMG_3613.JPG,40.689000,-74.043667,22.880661,2013:07:29 10:21:47


In [5]:
df_fotos_videos

,file_name,latitude,longitude,altitude,date_time
0,IMG_3600.JPG,40.646500,-73.789667,NaN,2013:07:28 19:02:00
1,IMG_3603.JPG,40.703667,-74.097000,NaN,2013:07:29 10:05:23
2,IMG_3605.JPG,40.690667,-74.042500,2.444101,2013:07:29 10:07:50
3,IMG_3607.JPG,40.690167,-74.042667,6.688098,2013:07:29 10:08:16
4,IMG_3613.JPG,40.689000,-74.043667,22.880661,2013:07:29 10:21:47
...,...,...,...,...,...
903,IMG_4931.JPG,37.809667,-122.410333,11.170458,2013:08:15 21:39:50
904,IMG_4932.JPG,37.778333,-122.391500,NaN,2013:08:15 22:51:22
905,IMG_4933.JPG,37.778333,-122.391500,NaN,2013:08:15 22:51:22
906,IMG_4934.JPG,37.628667,-122.400667,28.823656,2013:08:15 23:19:45


In [5]:
# Cargar el DataFrame
df = pd.read_csv('C:/Users/34670/Desktop/python/coast_to_coast/viaje_usa/data/metadata_singps.csv')
df.head()

,file_name,date_time
0,IMG_5457.JPG,2013:07:27 20:07:08
1,IMG_5460.JPG,2013:07:28 09:34:27
2,IMG_5461.JPG,2013:07:29 01:55:50
3,IMG_5464.JPG,2013:07:29 01:58:11
4,IMG_5465.JPG,2013:07:29 02:20:45


In [7]:
# Convertir la columna date_time a tipo datetime
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y:%m:%d %H:%M:%S')

# Ordenar el DataFrame por fecha
df = df.sort_values(by='date_time').reset_index(drop=True)

# Crear una nueva columna para almacenar la jornada
df['jornada'] = 0

# Obtener las fechas únicas en orden
fechas_unicas = df['date_time'].dt.date.unique()

# Asignar el número de jornada a cada fecha
for idx, fecha in enumerate(fechas_unicas):
    df.loc[df['date_time'].dt.date == fecha, 'jornada'] = f'dia_{idx + 1}'

# Mostrar el DataFrame con la nueva columna
df.head(10)


C:\Users\34670\AppData\Local\Temp\ipykernel_16336\1766395235.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'dia_1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['date_time'].dt.date == fecha, 'jornada'] = f'dia_{idx + 1}'


,file_name,date_time,jornada
0,IMG_5457.JPG,2013-07-27 20:07:08,dia_1
1,IMG_5460.JPG,2013-07-28 09:34:27,dia_2
2,IMG_5461.JPG,2013-07-29 01:55:50,dia_3
3,IMG_5464.JPG,2013-07-29 01:58:11,dia_3
4,IMG_5465.JPG,2013-07-29 02:20:45,dia_3
5,IMG_5467.JPG,2013-07-29 02:22:28,dia_3
6,IMG_5469.JPG,2013-07-29 02:26:44,dia_3
7,IMG_5474.JPG,2013-07-29 02:59:51,dia_3
8,IMG_5480.JPG,2013-07-29 03:05:52,dia_3
9,IMG_5483.JPG,2013-07-29 03:14:26,dia_3


In [8]:

# Supongamos que 'conteo_jornadas' es la serie obtenida anteriormente
conteo_jornadas = df['jornada'].value_counts()

# Convertir a DataFrame para facilitar el manejo
conteo_jornadas_df = conteo_jornadas.reset_index()
conteo_jornadas_df.columns = ['jornada', 'conteo']

# Extraer el número de la jornada y agregarlo como una nueva columna
conteo_jornadas_df['numero'] = conteo_jornadas_df['jornada'].str.extract('(\d+)', expand=False).astype(int)

# Ordenar el DataFrame por el número de la jornada
conteo_jornadas_df = conteo_jornadas_df.sort_values(by='numero').reset_index(drop=True)

# Eliminar la columna 'numero' si no la necesitas más
conteo_jornadas_df = conteo_jornadas_df.drop(columns='numero')

# Mostrar el DataFrame ordenado
conteo_jornadas_df


,jornada,conteo
0,dia_1,1
1,dia_2,1
2,dia_3,60
3,dia_4,57
4,dia_5,49
5,dia_6,25
6,dia_7,30
7,dia_8,17
8,dia_9,23
9,dia_10,78


In [1]:
import pandas as pd

In [95]:
df_1 = pd.read_csv('C:/Users/34670/Desktop/python/coast_to_coast/viaje_usa/data/metadata.csv')
df_2 = pd.read_csv('C:/Users/34670/Desktop/python/coast_to_coast/viaje_usa/data/metadata_singps.csv')

In [96]:
df_2

,file_name,date_time
0,IMG_5457.JPG,2013:07:27 20:07:08
1,IMG_5460.JPG,2013:07:28 09:34:27
2,IMG_5461.JPG,2013:07:29 01:55:50
3,IMG_5464.JPG,2013:07:29 01:58:11
4,IMG_5465.JPG,2013:07:29 02:20:45
...,...,...
632,IMG_9705.JPG,2013:08:16 02:49:01
633,IMG_9706.JPG,2013:08:16 02:49:44
634,IMG_9708.JPG,2013:08:16 04:44:55
635,IMG_9710.JPG,2013:08:16 04:45:34


In [97]:
# Convertir la columna date_time de df_1 a tipo datetime
df_1['date_time'] = pd.to_datetime(df_1['date_time'], format='%Y:%m:%d %H:%M:%S', errors='coerce')

# Convertir la columna date_time de df_2 a tipo datetime
df_2['date_time'] = pd.to_datetime(df_2['date_time'], format='%Y:%m:%d %H:%M:%S', errors='coerce')


In [98]:
import pytz
from datetime import timedelta

# Definir las zonas horarias
tz_spain = pytz.timezone('Europe/Madrid')
tz_us_eastern = pytz.timezone('US/Eastern')
tz_us_central = pytz.timezone('US/Central')
tz_us_mountain = pytz.timezone('US/Mountain')
tz_us_pacific = pytz.timezone('US/Pacific')

# Función para ajustar las horas según el rango de archivos
def ajustar_hora(row):
    dt = tz_spain.localize(row['date_time']) + timedelta(hours=1)
    if row['file_name'] in ['IMG_5457.JPG', 'IMG_5460.JPG']:
        return dt.replace(tzinfo=None)  # No cambiar la hora
    elif 'IMG_5461.JPG' <= row['file_name'] <= 'IMG_6388.JPG':
        return dt.astimezone(tz_us_eastern).replace(tzinfo=None)
    elif 'IMG_6440.JPG' <= row['file_name'] <= 'IMG_7000.JPG':
        return dt.astimezone(tz_us_central).replace(tzinfo=None)
    elif 'IMG_7004.JPG' <= row['file_name'] <= 'IMG_8396.JPG':
        return dt.astimezone(tz_us_mountain).replace(tzinfo=None)
    elif 'IMG_8398.JPG' <= row['file_name'] <= 'IMG_9713.JPG':
        return dt.astimezone(tz_us_pacific).replace(tzinfo=None)
    else:
        return dt.replace(tzinfo=None)

# Ajustar las horas en df_2
df_2['date_time'] = df_2.apply(ajustar_hora, axis=1)

# Mostrar el DataFrame con las horas ajustadas
df_2


,file_name,date_time
0,IMG_5457.JPG,2013-07-27 21:07:08
1,IMG_5460.JPG,2013-07-28 10:34:27
2,IMG_5461.JPG,2013-07-28 20:55:50
3,IMG_5464.JPG,2013-07-28 20:58:11
4,IMG_5465.JPG,2013-07-28 21:20:45
...,...,...
632,IMG_9705.JPG,2013-08-15 18:49:01
633,IMG_9706.JPG,2013-08-15 18:49:44
634,IMG_9708.JPG,2013-08-15 20:44:55
635,IMG_9710.JPG,2013-08-15 20:45:34


In [99]:
# Añadir columnas a df_2 con valores NaN
df_2['latitude'] = float('nan')
df_2['longitude'] = float('nan')
df_2['altitude'] = float('nan')


In [100]:
df_unido = pd.concat([df_1, df_2], ignore_index=True)
df_unido


,file_name,latitude,longitude,altitude,date_time
0,IMG_3600.JPG,40.646500,-73.789667,NaN,2013-07-28 19:02:00
1,IMG_3603.JPG,40.703667,-74.097000,NaN,2013-07-29 10:05:23
2,IMG_3605.JPG,40.690667,-74.042500,2.444101,2013-07-29 10:07:50
3,IMG_3607.JPG,40.690167,-74.042667,6.688098,2013-07-29 10:08:16
4,IMG_3613.JPG,40.689000,-74.043667,22.880661,2013-07-29 10:21:47
...,...,...,...,...,...
1540,IMG_9705.JPG,NaN,NaN,NaN,2013-08-15 18:49:01
1541,IMG_9706.JPG,NaN,NaN,NaN,2013-08-15 18:49:44
1542,IMG_9708.JPG,NaN,NaN,NaN,2013-08-15 20:44:55
1543,IMG_9710.JPG,NaN,NaN,NaN,2013-08-15 20:45:34


In [101]:
# Verificar cuántos valores NaT hay en date_time
nulos_date_time = df_2['date_time'].isnull().sum()
print(f"Valores NaT en 'date_time' antes de ordenar: {nulos_date_time}")

Valores NaT en 'date_time' antes de ordenar: 0


In [102]:
df_sinhora = df_unido[df_unido['date_time'].isnull()]
df_unido = df_unido.dropna(subset=['date_time'])

In [103]:
# Ordenar el DataFrame por la columna date_time de menor a mayor
df_unido = df_unido.sort_values(by='date_time').reset_index(drop=True)

df_unido


,file_name,latitude,longitude,altitude,date_time
0,IMG_5457.JPG,NaN,NaN,NaN,2013-07-27 21:07:08
1,IMG_5460.JPG,NaN,NaN,NaN,2013-07-28 10:34:27
2,IMG_3600.JPG,40.646500,-73.789667,NaN,2013-07-28 19:02:00
3,IMG_5461.JPG,NaN,NaN,NaN,2013-07-28 20:55:50
4,IMG_5464.JPG,NaN,NaN,NaN,2013-07-28 20:58:11
...,...,...,...,...,...
1514,IMG_4931.JPG,37.809667,-122.410333,11.170458,2013-08-15 21:39:50
1515,IMG_4932.JPG,37.778333,-122.391500,NaN,2013-08-15 22:51:22
1516,IMG_4933.JPG,37.778333,-122.391500,NaN,2013-08-15 22:51:22
1517,IMG_4935.JPG,37.628667,-122.400667,28.823656,2013-08-15 23:19:45


In [104]:
# Lista de nombres de archivo que queremos extraer
file_names = ['IMG_3603.JPG', 'IMG_3623.JPG', 'IMG_5508.JPG','IMG_5519.JPG', 'IMG_5508.JPG', 'IMG_5608.JPG']

# Filtrar las filas correspondientes a los nombres de archivo especificados
filas_especificas = df_unido.loc[df_unido['file_name'].isin(file_names)]

# Mostrar las filas extraídas
print(filas_especificas)


       file_name   latitude  longitude   altitude           date_time
14  IMG_3603.JPG  40.703667 -74.097000        NaN 2013-07-29 10:05:23
19  IMG_5508.JPG        NaN        NaN        NaN 2013-07-29 10:15:02
22  IMG_5519.JPG        NaN        NaN        NaN 2013-07-29 10:22:04
29  IMG_3623.JPG  40.688667 -74.043833  11.837721 2013-07-29 10:29:01
35  IMG_5608.JPG        NaN        NaN        NaN 2013-07-29 11:29:14


In [105]:
# Crear la columna 'coordenadas'
df_unido['coordenadas'] = df_unido.apply(lambda row: 'SI' if pd.notna(row['latitude']) and pd.notna(row['longitude']) else 'NO', axis=1)

# Mostrar las primeras filas para verificar
print(df_unido.head())


      file_name  latitude  longitude  altitude           date_time coordenadas
0  IMG_5457.JPG       NaN        NaN       NaN 2013-07-27 21:07:08          NO
1  IMG_5460.JPG       NaN        NaN       NaN 2013-07-28 10:34:27          NO
2  IMG_3600.JPG   40.6465 -73.789667       NaN 2013-07-28 19:02:00          SI
3  IMG_5461.JPG       NaN        NaN       NaN 2013-07-28 20:55:50          NO
4  IMG_5464.JPG       NaN        NaN       NaN 2013-07-28 20:58:11          NO


In [106]:
# Contar los valores de 'SI' y 'NO' en la columna 'coordenadas'
conteo_coordenadas = df_unido['coordenadas'].value_counts()

# Mostrar los resultados
print(conteo_coordenadas)


coordenadas
SI    835
NO    684
Name: count, dtype: int64


In [107]:
from datetime import timedelta

# Función para propagar coordenadas
def propagar_coordenadas(df):
    # Iterar sobre las filas que tienen 'SI' en la columna 'coordenadas'
    for idx, row in df[df['coordenadas'] == 'SI'].iterrows():
        # Obtener las coordenadas y el timestamp
        lat = row['latitude']
        lon = row['longitude']
        time = row['date_time']

        # Definir el rango de tiempo (3 minutos antes y después)
        time_start = time - timedelta(minutes=5)
        time_end = time + timedelta(minutes=5)

        # Propagar coordenadas a las filas dentro del rango de tiempo
        mask = (df['date_time'] >= time_start) & (df['date_time'] <= time_end) & (df['coordenadas'] == 'NO')
        df.loc[mask, 'latitude'] = lat
        df.loc[mask, 'longitude'] = lon

    return df

# Aplicar la función para propagar coordenadas
df_unido = propagar_coordenadas(df_unido)

# Mostrar las primeras filas para verificar
df_unido

,file_name,latitude,longitude,altitude,date_time,coordenadas
0,IMG_5457.JPG,NaN,NaN,NaN,2013-07-27 21:07:08,NO
1,IMG_5460.JPG,NaN,NaN,NaN,2013-07-28 10:34:27,NO
2,IMG_3600.JPG,40.646500,-73.789667,NaN,2013-07-28 19:02:00,SI
3,IMG_5461.JPG,NaN,NaN,NaN,2013-07-28 20:55:50,NO
4,IMG_5464.JPG,NaN,NaN,NaN,2013-07-28 20:58:11,NO
...,...,...,...,...,...,...
1514,IMG_4931.JPG,37.809667,-122.410333,11.170458,2013-08-15 21:39:50,SI
1515,IMG_4932.JPG,37.778333,-122.391500,NaN,2013-08-15 22:51:22,SI
1516,IMG_4933.JPG,37.778333,-122.391500,NaN,2013-08-15 22:51:22,SI
1517,IMG_4935.JPG,37.628667,-122.400667,28.823656,2013-08-15 23:19:45,SI


In [108]:
# Contar los valores NaN en la columna 'latitude'
conteo_nan_latitude = df_unido['latitude'].isnull().sum()

# Contar los valores no NaN en la columna 'latitude'
conteo_no_nan_latitude = df_unido['latitude'].notnull().sum()

# Mostrar los resultados
print(f"Valores NaN en 'latitude': {conteo_nan_latitude}")
print(f"Valores no NaN en 'latitude': {conteo_no_nan_latitude}")


Valores NaN en 'latitude': 494
Valores no NaN en 'latitude': 1025


In [109]:
import pandas as pd

# Asegurarse de que todos los valores en date_time son válidos datetime
df_unido['date_time'] = pd.to_datetime(df_unido['date_time'], errors='coerce')

# Ordenar el DataFrame por la columna date_time de menor a mayor
df_unido = df_unido.sort_values(by='date_time').reset_index(drop=True)

# Crear la columna 'dia'
df_unido['dia'] = (df_unido['date_time'].dt.date).factorize()[0] + 1

# Mostrar las primeras filas del DataFrame para verificar
print(df_unido.head(20))


       file_name   latitude  longitude  altitude           date_time  \
0   IMG_5457.JPG        NaN        NaN       NaN 2013-07-27 21:07:08   
1   IMG_5460.JPG        NaN        NaN       NaN 2013-07-28 10:34:27   
2   IMG_3600.JPG  40.646500 -73.789667       NaN 2013-07-28 19:02:00   
3   IMG_5461.JPG        NaN        NaN       NaN 2013-07-28 20:55:50   
4   IMG_5464.JPG        NaN        NaN       NaN 2013-07-28 20:58:11   
5   IMG_5465.JPG        NaN        NaN       NaN 2013-07-28 21:20:45   
6   IMG_5467.JPG        NaN        NaN       NaN 2013-07-28 21:22:28   
7   IMG_5469.JPG        NaN        NaN       NaN 2013-07-28 21:26:44   
8   IMG_5474.JPG        NaN        NaN       NaN 2013-07-28 21:59:51   
9   IMG_5480.JPG        NaN        NaN       NaN 2013-07-28 22:05:52   
10  IMG_5483.JPG        NaN        NaN       NaN 2013-07-28 22:14:26   
11  IMG_5485.JPG        NaN        NaN       NaN 2013-07-29 08:34:43   
12  IMG_5486.JPG        NaN        NaN       NaN 2013-07-29 09:0

In [110]:
df_unido

,file_name,latitude,longitude,altitude,date_time,coordenadas,dia
0,IMG_5457.JPG,NaN,NaN,NaN,2013-07-27 21:07:08,NO,1
1,IMG_5460.JPG,NaN,NaN,NaN,2013-07-28 10:34:27,NO,2
2,IMG_3600.JPG,40.646500,-73.789667,NaN,2013-07-28 19:02:00,SI,2
3,IMG_5461.JPG,NaN,NaN,NaN,2013-07-28 20:55:50,NO,2
4,IMG_5464.JPG,NaN,NaN,NaN,2013-07-28 20:58:11,NO,2
...,...,...,...,...,...,...,...
1514,IMG_4931.JPG,37.809667,-122.410333,11.170458,2013-08-15 21:39:50,SI,20
1515,IMG_4932.JPG,37.778333,-122.391500,NaN,2013-08-15 22:51:22,SI,20
1516,IMG_4933.JPG,37.778333,-122.391500,NaN,2013-08-15 22:51:22,SI,20
1517,IMG_4935.JPG,37.628667,-122.400667,28.823656,2013-08-15 23:19:45,SI,20


In [112]:
df_unido.to_excel('C:/Users/34670/Desktop/python/coast_to_coast/viaje_usa/data/df_unido.xlsx', index=False)